### 1. Create a constant value @property

In [1]:
class Circle():
  def __init__(self, radius):
    self.radius = radius
    self.pi = 3.14  
    
  def area(self):
    return self.pi * self.radius * self.radius 
  
  def perimeter(self):
    return 2 * self.pi * self.radius

In [2]:
c = Circle(4)
c.pi = 5
c.area()

80

In [3]:
# fix this with @property
class Circle():
  def __init__(self, radius):
    self.radius = radius
   
  @property
  def pi(self):
    return 3.14

  def area(self):
    return self.pi * self.radius * self.radius 
  
  def perimeter(self):
    return 2 * self.pi * self.radius

In [4]:
c = Circle(4)
c.pi

3.14

In [5]:
c.perimeter()

25.12

### 2. Multiple Class constructors

In [6]:
class Date():
  def __init__(self, day, month, year):
    self.day = day
    self.month = month
    self.year = year
    
  @classmethod
  def fromString(cls, s):
   day = int(s[:2])
   month = int(s[3:5])
   year = int(s[6:])
   return cls(day, month, year) # Return a new object

In [7]:
d = Date.fromString("21/07/2020")
print(d.day, d.month, d.year)

21 7 2020


### 3. Creating Enum

In [8]:
from enum import Enum

class WeekDay(Enum):
  Monday = 1
  Tuesday = 2
  Wednesday = 3
  Thursday = 4
  Friday = 5
  Saturday = 6
  Sunday = 7

In [9]:
day = WeekDay.Friday

In [10]:
print(day.name, day.value)

Friday 5


### 4. Backwords Iterators

In [11]:
class Backward():
  def __init__(self, data):
    self.data = data # The list we want to iterate 
    self.index = len(self.data)  

  def __iter__(self):
    return self
  
  def __next__(self):
    if(self.index == 0):
      raise StopIteration
    else:
      self.index -= 1
      return self.data[self.index]

In [12]:
bw = Backward([1,2,3,4,5])
for elem in bw:
  print(elem, end=' ')

5 4 3 2 1 

### 5. Accessing a class as a list
Here is a really easy example (it is a list whose size cannot be changed, and whose index start at 1):

In [13]:
class MyList():
  def __init__(self, dimension):
    self.l = [0 for i in range(dimension)]
  
  def __getitem__(self, idx):
    return self.l[idx-1]
  
  def __setitem__(self, idx, data):
    self.l[idx-1]=data


In [14]:
ml = MyList(5)
ml[1] = 50 # Set the first element of ml to 50
ml[2] = 100 # Set the second element of ml to 100
x = ml[1] # x is now 50

In [15]:
x

50

#### 5.1 __getitem__ and __setitem__ how to

In [16]:
# Without using __getitem__ we would have a class like this :
class Building(object):
     def __init__(self, floors):
         self._floors = [None]*floors
     def occupy(self, floor_number, data):
          self._floors[floor_number] = data
     def get_floor_data(self, floor_number):
          return self._floors[floor_number]

building1 = Building(4) # Construct a building with 4 floors
building1.occupy(0, 'Reception')
building1.occupy(1, 'ABC Corp')
building1.occupy(2, 'DEF Inc')
print( building1.get_floor_data(2) )

DEF Inc


In [17]:
# We could however use __getitem__ (and its counterpart __setitem__) to make the usage of the Building class 'nicer'.
class Building(object):
     def __init__(self, floors):
         self._floors = [None]*floors
     def __setitem__(self, floor_number, data):
          self._floors[floor_number] = data
     def __getitem__(self, floor_number):
          return self._floors[floor_number]

building1 = Building(4) # Construct a building with 4 floors
building1[0] = 'Reception'
building1[1] = 'ABC Corp'
building1[2] = 'DEF Inc'
print( building1[2] )

DEF Inc


#### 5.2 magic method __getattr__ 

In [18]:
class Person(object):
    def __init__(self, first_name, last_name, address):
        self.first_name = first_name
        self.last_name = last_name
        self.address = address

    @classmethod
    def create_using_full_name(cls, full_name, address):
        name = full_name.split(' ')
        return cls(name[0], name[1], address)

In [19]:
person1 = Person('Henry', 'Thiery', 'France')
person2 = Person('Xaxi', 'Hernandez', 'Spain')
person3 = Person('Erling', 'Haaland', 'Sweden')

In [20]:
class Employees(object):
    def __init__(self):
        self.employees_list = []

    def __len__(self):
        return len(self.employees_list)

    def __getitem__(self, item):
        return self.employees_list[item]

    @classmethod
    def add_employee_to_list(cls, list_of_emps=[]):
        self = cls()
        for emp in list_of_emps:
            self.employees_list.append(emp)
        return self

    def print_emp_name(self):
        return [person.first_name + ' ' + person.last_name for person in self.employees_list]

In [21]:
employees = Employees.add_employee_to_list([person1, person2, person3])
employees.print_emp_name()
# [‘HenryThiery’, ‘XaxiHernandez’, ‘ErlingHaaland’]

['Henry Thiery', 'Xaxi Hernandez', 'Erling Haaland']

In [22]:
len(employees)

3

In [23]:
employees[1]

In [24]:
employees[1].first_name

'Xaxi'

Here employees is a List like object but not a List. Suppose we want to have append methods of List object available in Employees object we need to inherit from List object and override append method in the Employee class definition which is tight coupling and difficult to maintain if we want to support more methods from List object.

In [25]:
# we can define a magic method __getattr__ which intercepts the ‘.’ 
# when we call an instance method and we can redirect it to the appropriate class 
# using the getattr method which we saw in the previous post.
class Employees(object):
    def __init__(self):
        self.employees_list = []

    def __len__(self):
        return len(self.employees_list)

    def __getitem__(self, item):
        return self.employees_list[item]

    def __getattr__(self, item):
        return getattr(self.employees_list, item)

    @classmethod
    def add_employee_to_list(cls, list_of_emps=[]):
        self = cls()
        for emp in list_of_emps:
            self.employees_list.append(emp)
        return self
    
    def print_emp_name(self):
        return [person.first_name + person.last_name for person in self.employees_list]

In [26]:
# Now the employee object magically has the methods of a List object, 
# it is actually being redirected to the main class because 
# we had intercepted the ‘.’ using getattr. Now onto our second problem.
person4 = Person('Zhefu', 'Fan', 'Canada')
employees = Employees.add_employee_to_list([person1, person2, person3])
employees.append(person4)


In [27]:
# employees.sort()

In [28]:
employees.index(person2)

1

In [29]:
for employee in employees:
#     print(employee.first_name + ' ' + employee.last_name + ':' + employee.address)
    print(f'{employee.first_name} {employee.last_name}: {employee.address}')

Henry Thiery: France
Xaxi Hernandez: Spain
Erling Haaland: Sweden
Zhefu Fan: Canada


### 6 Mixins

In [30]:
class Person(object):
    def __init__(self, name, age, sex, profession):
        self.name = name
        self.age = age
        self.sex = sex
        self.profession = profession

In [31]:
person1 = Person('Henry Thiery', '42', 'Male', 'Footballer')
person2 = Person('Xaxi Hernandez', '35', 'Female', 'Footballer')
person3 = Person('Erling Haaland', '21', 'Male', 'Footballer')

In [32]:
persons = [person1, person2, person3]

In [33]:
def print_table(objects, attrs):
    for attr_name in attrs:
        print(attr_name, end='\t')
    print()
    for obj in objects:
        for attr_name in attrs:
            print(getattr(obj, attr_name), end="\t")
        print()

In [34]:
def print_table(objects, attrs, format):
    format.heading(attrs)
    for obj in objects:
        rows = [str(getattr(obj, attr_name)) for attr_name in attrs]
        format.contents(rows)

In [35]:
class Formatter(object):
    def __init__(self, save_file=None):
        if save_file:
            self.save_file = save_file

    def heading(self, headers):
        raise NotImplementedError

    def contents(self, rows):
        raise NotImplementedError

class TextFormatter(
    Formatter):  # if i want to support width param or add a feature then I would need to init parent and create my var

    def __init__(self, save_file=None, width=20):
        self.width = width
        super().__init__(save_file=save_file)

    def heading(self, headers):
        for header in headers:
            print(header, end='\t')
        print()

    def contents(self, rows):
        print()
        for row in rows:
            print({}.format(row), end="\t")

class CVSFormatter(Formatter):
    def heading(self, headers):
        for header in headers:
            print(header, end=',')
        print()

    def contents(self, rows):
        for row in rows:
            print('{}'.format(row), end=",")
        print()

class HTMLFormatter(Formatter):
    def heading(self, headers):
        for header in headers:
            print('<h>{}</h>'.format(header), end=' ')
        print()

    def contents(self, rows):
        for row in rows:
            print('<t>{}</t>'.format(row), end=" ")
        print()
        
persons = [person1, person2, person3]

In [36]:
# Creating a formatter
formatter = CVSFormatter()
print_table(persons, ['name', 'age', 'profession'], format=formatter)

name,age,profession,
Henry Thiery,42,Footballer,
Xaxi Hernandez,35,Footballer,
Erling Haaland,21,Footballer,


In [37]:
# Creating another formatter
formatter = HTMLFormatter()
print_table(persons, ['name', 'age', 'profession'], format=formatter)

<h>name</h> <h>age</h> <h>profession</h> 
<t>Henry Thiery</t> <t>42</t> <t>Footballer</t> 
<t>Xaxi Hernandez</t> <t>35</t> <t>Footballer</t> 
<t>Erling Haaland</t> <t>21</t> <t>Footballer</t> 


## Mixin

In [38]:
class QuoterMixin(object):
    def contents(self, rows):
        quoted = ['"{}"'.format(text) for text in rows]
        super().contents(quoted)

class Formatter(QuoterMixin, CVSFormatter):
    pass

formatter = Formatter()
person1 = Person('Thiery Henry', '42', 'M', 'Footballer')
person2 = Person('Hernandez Xaxi', '35', 'M', 'Footballer')
person3 = Person('Erling Haaland', '21', 'M', 'Footballer')

persons = [person1, person2, person3]
print_table(persons, ['name', 'age', 'profession'], format=formatter)
formatter = CVSFormatter()
print_table(persons, ['name', 'age', 'profession'], format=formatter)
formatter = HTMLFormatter()
print_table(persons, ['name', 'age', 'profession'], format=formatter)

name,age,profession,
"Thiery Henry","42","Footballer",
"Hernandez Xaxi","35","Footballer",
"Erling Haaland","21","Footballer",
name,age,profession,
Thiery Henry,42,Footballer,
Hernandez Xaxi,35,Footballer,
Erling Haaland,21,Footballer,
<h>name</h> <h>age</h> <h>profession</h> 
<t>Thiery Henry</t> <t>42</t> <t>Footballer</t> 
<t>Hernandez Xaxi</t> <t>35</t> <t>Footballer</t> 
<t>Erling Haaland</t> <t>21</t> <t>Footballer</t> 
